In [5]:
from sqlalchemy import create_engine
import pandas as pd
import getpass  # To get the password without showing the input
password = getpass.getpass()



········


In [6]:
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)
data_may = pd.read_sql_query('SELECT * FROM sakila.rental_may', engine)
data_june = pd.read_sql_query('SELECT * FROM sakila.rental_june', engine)


In [7]:
data_may.head()



,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53
2,3,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39,1,2006-02-15 21:30:53
3,4,2005-05-24 23:04:41,2452,333,2005-06-03 01:43:41,2,2006-02-15 21:30:53
4,5,2005-05-24 23:05:21,2079,222,2005-06-02 04:33:21,1,2006-02-15 21:30:53


In [8]:
data_june.head()



,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1158,2005-06-14 22:53:33,1632,416,2005-06-18 21:37:33,2,2006-02-15 21:30:53
1,1159,2005-06-14 22:55:13,4395,516,2005-06-17 02:11:13,1,2006-02-15 21:30:53
2,1160,2005-06-14 23:00:34,2795,239,2005-06-18 01:58:34,2,2006-02-15 21:30:53
3,1161,2005-06-14 23:07:08,1690,285,2005-06-21 17:12:08,1,2006-02-15 21:30:53
4,1162,2005-06-14 23:09:38,987,310,2005-06-23 22:00:38,1,2006-02-15 21:30:53


In [9]:
engine.execute("USE sakila")
query = "SELECT count(rental_id) as number_rentals_may, customer_id \
FROM sakila.rental_may \
GROUP BY customer_id"
cus_may_df = pd.read_sql_query(query, engine)
cus_may_df.head()



,number_rentals_may,customer_id
0,2,130
1,1,459
2,3,408
3,1,333
4,5,222


In [10]:
engine.execute("USE sakila")
query1 = "SELECT count(rental_id) as number_rentals_june, customer_id \
FROM sakila.rental_june \
GROUP BY customer_id"
cus_june_df = pd.read_sql_query(query1, engine)
cus_june_df.head()





,number_rentals_june,customer_id
0,5,416
1,6,516
2,5,239
3,3,285
4,6,310


In [11]:
merged_frames = pd.merge(cus_may_df,cus_june_df,on='customer_id',how='outer')



In [12]:
merged_frames.head()




,number_rentals_may,customer_id,number_rentals_june
0,2.0,130,6.0
1,1.0,459,7.0
2,3.0,408,3.0
3,1.0,333,4.0
4,5.0,222,2.0


In [13]:
merged_frames = merged_frames[['customer_id', 'number_rentals_may','number_rentals_june']]



In [14]:
merged_frames.head()



,customer_id,number_rentals_may,number_rentals_june
0,130,2.0,6.0
1,459,1.0,7.0
2,408,3.0,3.0
3,333,1.0,4.0
4,222,5.0,2.0


In [15]:
merged_frames['number_rentals_may'] = merged_frames['number_rentals_may'].fillna(0)



In [16]:
merged_frames['number_rentals_june'] = merged_frames['number_rentals_june'].fillna(0)



In [17]:
print("The dataframe has {} rows and {} columns".format(merged_frames.shape[0],merged_frames.shape[1]))
print()
print("The data types of each column of the dataframe are:")
print(merged_frames.dtypes)
print()



The dataframe has 598 rows and 3 columns

The data types of each column of the dataframe are:
customer_id              int64
number_rentals_may     float64
number_rentals_june    float64
dtype: object



In [18]:
def check_cus_number(row):
    if row['number_rentals_may'] > row['number_rentals_june']:
        return 5
    elif row['number_rentals_may'] < row['number_rentals_june']:
        return 6
    else:
        return 0



In [19]:
merged_frames ['comparison'] = merged_frames.apply (lambda row: check_cus_number(row), axis=1)



In [20]:
merged_frames.head()


,customer_id,number_rentals_may,number_rentals_june,comparison
0,130,2.0,6.0,6
1,459,1.0,7.0,6
2,408,3.0,3.0,0
3,333,1.0,4.0,6
4,222,5.0,2.0,5
